# spectrum demo

## get an emission measure file, run mspectrum, save the data and work with the restored data

In [ ]:
import os
import pickle
import numpy as np
import matplotlib.pyplot as plt
import ChiantiPy.core as ch
import ChiantiPy.tools.util as chutil
import ChiantiPy.tools.io as chio
import ChiantiPy.tools.filters as chfilters

In [ ]:
autoreload 3

In [ ]:
matplotlib qt

## get an emission measure file

## these are in XUVTOP/em

In [ ]:
emDir = os.path.join(os.environ['XUVTOP'], 'em')

In [ ]:
emList = os.listdir(emDir)

In [ ]:
for idx, em in enumerate(emList):
    print(' %i  %s'%(idx, em))

## at present, there are only 2 files, a quiet sun EM and an active EM

## I will pick the active region EM

In [ ]:
emAR = chio.emRead(emList[1])

In [ ]:
emAR.keys()

In [ ]:
for idx, temp in enumerate(emAR['temperature']):
    print(' %i  %10.2e  %10.2e  %10.2e'%(idx, temp, emAR['density'][idx], emAR['em'][idx]))

## this is similar to one of the quick start examples but uses the EM distribution

In [ ]:
temp = emAR['temperature']
dens = emAR['density']
em = emAR['em']

In [ ]:
wvlRange = [300.,400]

In [ ]:
dwvl = 0.01
nwvl = int((wvlRange[1] - wvlRange[0])/dwvl + 1)

In [ ]:
wvl = np.linspace(wvlRange[0], wvlRange[1], nwvl)

### will use mspectrum to use multiple cores/processors

In [ ]:
cores = 4

In [ ]:
sp = ch.mspectrum(temp, dens, wvl, filter = (chfilters.gaussian,3.*dwvl), em = em, doContinuum=0, minAbund=1.e-5,
                  proc=cores)

In [ ]:
sp.spectrumPlot(integrated=True)

## save the calculations to a pickle file

In [ ]:
saveName = 'spectrum_demo.pkl'

In [ ]:
sp.saveData(saveName)

In [ ]:
sp.Intensity.keys()

In [ ]:
sp.Em.size

### index must be 0 through spectrumDict['em'].size - 1

In [ ]:
sp.intensityPlot(index=2, wvlRange=wvlRange, top=20, linLog='lin')

In [ ]:
sp.intensityPlot(index=2, wvlRange=[330., 370.], top=20, linLog='lin')

In [ ]:
sp.intensityPlot(index=2, wvlRange=[340.,370.], top=20, linLog='lin')

## restore the calculations from the pickle file

In [ ]:
with open(saveName,'rb') as inpt:
    spectrumDict = pickle.load(inpt)

In [ ]:
spectrumDict.keys()

### examine what is in spectrumDict

In [ ]:
for akey in spectrumDict.keys():
    print(' key:  %s'%(akey))

In [ ]:
spectrumDict['abundanceName']

In [ ]:
spectrumDict['abundAll'].shape

In [ ]:
spectrumDict['defaults'].keys()

In [ ]:
spectrumDict['em'].shape

In [ ]:
spectrumDict['intensity'].keys()

In [ ]:
spectrumDict['intensity']['intensity'].shape

In [ ]:
spectrumDict['intensity']['integrated'].shape

In [ ]:
len(spectrumDict['intensity']['ionS'])

In [ ]:
spectrumDict['intensity']['ionS'][0]

In [ ]:
spectrumDict['nTemp']

In [ ]:
spectrumDict['nDens']

In [ ]:
spectrumDict['nTempDens']

In [ ]:
min(spectrumDict['intensity']['wvl'])

In [ ]:
max(spectrumDict['intensity']['wvl'])

## make another plot of lines calculated in spectrum/mspectrum

## there is more flexibility when working with the saveData dict

In [ ]:
fs = 10  # fontsize
figsize = [7., 5.]

In [ ]:
fig, ax = plt.subplots(figsize=figsize)

In [ ]:
ax.set_xlim(wvlRange)

## it is best to only plot those wavelength in the wvlRange

In [ ]:
index = chutil.between(spectrumDict['intensity']['wvl'], wvlRange)

In [ ]:
len(index)

In [ ]:
spectrumDict['intensity']['integrated'][index].size

### just plot the most intense lines, minIntensity could be set to zero

In [ ]:
minIntensity = spectrumDict['intensity']['integrated'][index].max()/20.

In [ ]:
' %10.3e'%(minIntensity)

In [ ]:
for idx in index:
    wvl = spectrumDict['intensity']['wvl'][idx]    
    integrated = spectrumDict['intensity']['integrated'][idx]
    if integrated > minIntensity:
        ax.plot([wvl, wvl] , [0., integrated])

In [ ]:
ylim = ax.get_ylim()

In [ ]:
ylim

In [ ]:
ax.set_ylim([0., 1.5*ylim[1]])

In [ ]:
for idx in index:
    wvl = spectrumDict['intensity']['wvl'][idx]    
    lbl = spectrumDict['intensity']['ionS'][idx] + ' %7.3f'%(wvl)
    if spectrumDict['intensity']['integrated'][idx] > minIntensity:
        ax.text(wvl, 1.2*spectrumDict['intensity']['integrated'][idx], lbl, va='bottom', ha='center', rotation='vertical', fontsize=fs)

In [ ]:
ax.set_xlabel(spectrumDict['xlabel'], fontsize=16)

In [ ]:
ax.set_ylabel(spectrumDict['ylabel'], fontsize=16)

In [ ]:
fig.tight_layout()

### looks kind of a mess, look at a narrower wavelength range

In [ ]:
fig, ax = plt.subplots(figsize=figsize)

In [ ]:
wvlRange = [340.,370.]

In [ ]:
ax.set_xlim(wvlRange)

In [ ]:
index = chutil.between(spectrumDict['intensity']['wvl'], wvlRange)

In [ ]:
len(index)

In [ ]:
spectrumDict['intensity']['integrated'][index].size

### just plot the most intense lines, minIntensity could be set to zero

In [ ]:
minIntensity = spectrumDict['intensity']['integrated'][index].max()/20.

In [ ]:
' %10.3e'%(minIntensity)

In [ ]:
for idx in index:
    wvl = spectrumDict['intensity']['wvl'][idx]    
    integrated = spectrumDict['intensity']['integrated'][idx]
    if integrated > minIntensity:
        ax.plot([wvl, wvl] , [0., integrated])

In [ ]:
ylim = ax.get_ylim()

In [ ]:
ylim

In [ ]:
ax.set_ylim([0., 1.5*ylim[1]])

In [ ]:
for idx in index:
    wvl = spectrumDict['intensity']['wvl'][idx]    
    lbl = spectrumDict['intensity']['ionS'][idx] + ' %7.3f'%(wvl)
    if spectrumDict['intensity']['integrated'][idx] > minIntensity:
        ax.text(wvl, 1.2*spectrumDict['intensity']['integrated'][idx], lbl, va ='bottom', ha='center',
                rotation='vertical', fontsize=fs)

In [ ]:
ax.set_xlabel(spectrumDict['xlabel'], fontsize=16)

In [ ]:
ax.set_ylabel(spectrumDict['ylabel'], fontsize=16)

In [ ]:
fig.tight_layout()

# plot the intensities for a given temperature

In [ ]:
for it, temp in enumerate(spectrumDict['temperature']):
    print('it:  %2i  temperature  %10.2e'%(it,temp))

## pick a temperature of 1.12e+6 K

In [ ]:
tIndex = 1

In [ ]:
fig, ax = plt.subplots(figsize=figsize)

In [ ]:
ax.set_xlim(wvlRange)

In [ ]:
index = chutil.between(spectrumDict['intensity']['wvl'], wvlRange)

In [ ]:
len(index)

In [ ]:
spectrumDict['intensity']['integrated'][index].size

### just plot the most intense lines, minIntensity could be set to zero

In [ ]:
minIntensity = spectrumDict['intensity']['intensity'][tIndex, index].max()/20.

In [ ]:
' %10.3e'%(minIntensity)

In [ ]:
for idx in index:
    wvl = spectrumDict['intensity']['wvl'][idx]    
    integrated = spectrumDict['intensity']['intensity'][tIndex, idx]
    if integrated > minIntensity:
        ax.plot([wvl, wvl] , [0., integrated])

In [ ]:
ylim = ax.get_ylim()

In [ ]:
ylim

In [ ]:
ax.set_ylim([0., 1.5*ylim[1]])

In [ ]:
for idx in index:
    wvl = spectrumDict['intensity']['wvl'][idx]    
    lbl = spectrumDict['intensity']['ionS'][idx] + ' %7.3f'%(wvl)
    if spectrumDict['intensity']['intensity'][tIndex, idx] > minIntensity:
        ax.text(wvl, 1.2*spectrumDict['intensity']['intensity'][tIndex,idx], lbl, va='bottom', ha='center',
                rotation='vertical', fontsize=fs)

In [ ]:
ax.set_xlabel(spectrumDict['xlabel'], fontsize=16)

In [ ]:
ax.set_ylabel(spectrumDict['ylabel'], fontsize=16)

In [ ]:
dstr = ' -  Density = %10.2e (cm$^{-3}$)' % spectrumDict['eDensity'][tIndex]
tstr = 'T = %10.2e (K)' % spectrumDict['temperature'][tIndex]
emstr = ' em = %10.2e' % spectrumDict['em'][tIndex]

In [ ]:
ax.set_title(tstr + dstr + '\n' + emstr, fontsize=14)

In [ ]:
fig.tight_layout()